In [27]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from pathlib import Path
import os
import pickle
from pathlib import Path
import os
from tensorflow import keras
from sklearn.metrics import mean_squared_error, mean_absolute_error

plt.style.use("dark_background")
plt.rcParams["figure.figsize"] = (30, 10)
plt.rcParams["figure.dpi"] = 600


In [28]:
TICKER = "BTC"
MODEL_VERSION = 0.2
MODEL_NAME = "lstm"

In [29]:
load_path = Path(os.path.abspath("")).parents[1] / "data" / "preprocessed"
models_path = Path(os.path.abspath("")).parents[1] / "models" / "lstm" / "versions"

with open(load_path / f"data_{TICKER}.pickle", "rb") as handle:
    data = pickle.load(handle)

with open(load_path / f"scalers_{TICKER}.pickle", "rb") as handle:
    scalers = pickle.load(handle)

In [30]:
from datetime import datetime
datetime.now().second

35

In [31]:
data["X_list_train"].shape

(33362, 60, 37)

In [32]:
model = keras.models.load_model(models_path / f"{MODEL_NAME}_{MODEL_VERSION}")

In [33]:
preds = model.predict(data["X_list_test"])

In [34]:
backtest_path = Path(os.path.abspath("")).parents[1] / "backtesting"

In [35]:
actual_values = data["Y_preds_real_list_test"]

In [36]:
test_preds = scalers["y_scaler"].inverse_transform(preds)
test_true = scalers["y_scaler"].inverse_transform(actual_values)

In [37]:
with open(backtest_path / f"preds_{TICKER}_{MODEL_VERSION}.pickle", "wb") as handle:
            pickle.dump(test_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [38]:
mean_absolute_error(y_true=test_true, y_pred=test_preds)

80.69493962876574

In [39]:

plt.plot(
    test_preds.reshape(
        test_preds.shape[0],
    ),
    color="blue",
    label="Predicted price",
)
plt.plot(
    test_true.reshape(
        test_true.shape[0],
    ),
    color="red",
    label="Actual price",
)
plt.title(f"{TICKER} actual price vs predicted price")
plt.xlabel("Day number")
plt.ylabel("Valuation")
plt.legend()
plt.show()